In [7]:
import sys
import os
import re
import numpy as np
import pandas as pd

import mne
from mne_bids import write_raw_bids, BIDSPath


In [8]:
# set different path to load and save data
ds_root='/Volumes/hyijie_psy/CPP/data_high-level_3/speeded'
tgt_dir='..'

dir_preprocess_behavior=os.path.join(tgt_dir,'data','preprocessData','emotion','behavior')
if not os.path.exists(dir_preprocess_behavior):
    os.makedirs(dir_preprocess_behavior)

dir_preprocess_eeg=os.path.join(tgt_dir,'data','preprocessData','emotion','eeg')
if not os.path.exists(dir_preprocess_eeg):
    os.makedirs(dir_preprocess_eeg)

In [ ]:
# drop some trials of behavior according to events
sub_eeg_dir = [f'Emotion_sub{i}' for i in range(1,19)]
sub_behavior_dir = [f'Emotion_decision_sub{i}' for i in range(1,19)]

sub_excluded = []

for sub_eeg, sub_behavior in zip(sub_eeg_dir, sub_behavior_dir):
    #load eeg and behavior data
    raw = mne.io.read_raw_cnt(os.path.join(ds_root, 'Rawdata', sub_eeg+'.cnt'))
    behavior = pd.read_csv(os.path.join(ds_root, 'behavior', sub_behavior+'.dat'),header=None)

    # format eeg data into BIDS format
    raw.set_channel_types({'HEOG': 'eog',  
                           'VEOG': 'eog',
                           'A1': 'misc'}) 
    # resample
    raw.resample(500, npad="auto")
    # filter
    raw.filter(1, 30, fir_design='firwin', picks=['eeg'])  
    # re-reference
    raw.set_eeg_reference('average')
    # ica remove artifact
    ica = mne.preprocessing.ICA(n_components=30, random_state=97) # 这里用了30个成分
    ica.fit(raw) 
    ica.exclude = []                                                                                       
    eog_indices, eog_scores = ica.find_bads_eog(raw)                  
    ica.exclude = eog_indices 
    ica.apply(raw) 

    events, event_id = mne.events_from_annotations(raw)

    bids_root = dir_preprocess_eeg
    bids_path = BIDSPath(
        subject=sub_eeg.replace('Emotion_sub',''),
        task='emotion',  
        datatype='eeg',
        root=bids_root
    )

    # 保存到 BIDS 格式
    write_raw_bids(
        raw=raw,
        bids_path=bids_path,
        events=events,
        event_id=event_id,
        overwrite=True,
        allow_preload=True,
        format='BrainVision'  # 保持 BrainVision 格式（.vhdr/.vmrk/.eeg）
    )

    # find specific trials which lack markers for response
    # the logic: select stimuli onset and response makers first and find continious marker for stimuli onset
    ## find events_id first because they are different among subjects
    stim_event_key = None
    event_keys = []
    for key, value in event_id.items():
        if '111' in key:  # '111' is marker for stimuli onset
            stim_event_key = value
        if '111' in key or '101' in key or '102' in key :  # '101' and '102' are response markers
            event_keys.append(value)
    
    ## select all events on stimuli onset and response
    events_select = events[np.isin(events[:, 2], event_keys)]

    ## if the number of stimuli onset marker is different with bahevior data, we can not use the dataset
    test4trial = np.sum(events_select[:, 2] == stim_event_key)
    if test4trial != behavior.shape[0]:
        sub_excluded.append(sub_eeg)
        print(f"Skipping subject {sub_eeg}: test4trial count ({test4trial}) is less than behavior rows ({behavior.shape[0]})")
        continue 
    
    ## index useless trials
    stim_index = np.where(events_select[:,2] == stim_event_key)[0]
    data_lacked = []
    for i in range(events_select.shape[0] - 1):
        if events_select[i,2] == stim_event_key and events_select[i+1,2] == stim_event_key:
            first_index = i
            rank = np.sum(stim_index <= first_index)
            data_lacked.append(rank)
    
    ## get dropped behavior data
    behavior_drop = behavior.drop(data_lacked, axis=0).reset_index(drop=True)
    behavior_drop.to_csv(os.path.join(dir_preprocess_behavior,sub_eeg+'.csv'), header=False, index=False)


sub_excluded = pd.DataFrame(sub_excluded)
sub_excluded.to_csv(os.path.join(dir_preprocess_behavior,'sub_excluded.csv'), header=False, index=False)


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_13251/3761815757.py:9: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = mne.io.read_raw_cnt(os.path.join(ds_root, 'Rawdata', sub_eeg+'.cnt'))
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_13251/3761815757.py:13: RuntimeWarning: The unit for channel(s) A1 has changed from V to NA.
  raw.set_channel_types({'HEOG': 'eog',


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Fitting ICA to data using 30 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.2s finished


Selecting by number: 30 components
Fitting ICA took 6.1s.
Using EOG channels: HEOG, VEOG
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)



/Users/hyijie/Desktop/桌面文件夹/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/scipy/linalg/_basic.py:1622: RuntimeWarning: divide by zero encountered in matmul
  B = (u @ vh[:rank]).conj().T
/Users/hyijie/Desktop/桌面文件夹/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/scipy/linalg/_basic.py:1622: RuntimeWarning: overflow encountered in matmul
  B = (u @ vh[:rank]).conj().T
/Users/hyijie/Desktop/桌面文件夹/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/scipy/linalg/_basic.py:1622: RuntimeWarning: invalid value encountered in matmul
  B = (u @ vh[:rank]).conj().T
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_13251/3761815757.py:24: RuntimeWarning: Using n_components=30 (resulting in n_components_=30) may lead to an unstable mixing matrix estimation because the ratio between the largest (12) and smallest (1.8e-29) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 26
  ica.fit(raw)
/Users/hyijie/Desktop/桌面文件夹

...

... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 sam

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Applying ICA to Raw instance
    Transforming to ICA space (30 components)
    Zeroing out 3 ICA components
    Projecting back using 30 PCA components
Used Annotations descriptions: [np.str_('101'), np.str_('102'), np.str_('103'), np.str_('11'), np.str_('111'), np.str_('112'), np.str_('12'), np.str_('121'), np.str_('122'), np.str_('123'), np.str_('13'), np.str_('14'), np.str_('15'), np.str_('16'), np.str_('17'), np.str_('21'), np.str_('22'), np.str_('23'), np.str_('24'), np.str_('25'), np.str_('26'), np.str_('27')]
Writing '../data/preprocessData/emotion/eeg/participants.tsv'...
Writing '../data/preprocessData/emotion/eeg/participants.json'...
Writing '../data/preprocessData/emotion/eeg/sub-18/eeg/sub-18_space-CapTrak_electrodes.tsv'...
Writing '../data/preprocessData/emotion/eeg/sub-18/eeg/sub-18_space-CapTrak_coordsystem.json'...
Used Annotations descriptions: [np.str_('101'), np.str_('102'), np.str_('103'), np.str_('11'), np.str_('111'), np.str_('112'), np.str_('12'), np.str_('121'

/Users/hyijie/Desktop/桌面文件夹/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/mne/preprocessing/ica.py:2401: RuntimeWarning: divide by zero encountered in matmul
  mixing = pca_components.T @ mixing
/Users/hyijie/Desktop/桌面文件夹/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/mne/preprocessing/ica.py:2401: RuntimeWarning: overflow encountered in matmul
  mixing = pca_components.T @ mixing
/Users/hyijie/Desktop/桌面文件夹/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/mne/preprocessing/ica.py:2401: RuntimeWarning: invalid value encountered in matmul
  mixing = pca_components.T @ mixing
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_13251/3761815757.py:41: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_13251/3761815757.py:41: RuntimeWarning: Encountered data in "int" format. Converting to float32.
  write_raw_bids(
/Users/hyijie/Desktop/桌面文件夹/Lab/CPP_Multiv

Writing '../data/preprocessData/emotion/eeg/sub-18/sub-18_scans.tsv'...
Wrote ../data/preprocessData/emotion/eeg/sub-18/sub-18_scans.tsv entry with eeg/sub-18_task-emotion_eeg.vhdr.
